In [28]:
import os
import numpy as np
import pandas as pd

In [3]:
path = './data/diamonds_20200620.csv'
df = pd.read_csv(path)
df.head()

,carat,clarity,color,culet,cut,date,dateSet,depth,detailsPageUrl,fluorescence,...,shapeCode,shapeName,skus,sold,strikethroughPrice,symmetry,table,v360BaseUrl,visualizationImageUrl,willArriveForHoliday
0,1.50,SI1,H,None,Ideal,Jun 10,Jun 11,59.9,./diamond-details/LD13542671,None,...,RD,Round,LD13542671,False,NaN,Excellent,60.0,https://bluenile.v360.in/51/imaged/gia-2205795...,https://bnsec.bluenile.com/bnsecure/diamondvis...,False
1,1.51,VS2,H,None,Ideal,Jun 22,Jun 23,62.7,./diamond-details/LD13873802,None,...,RD,Round,LD13873802,False,NaN,Excellent,57.0,NaN,NaN,False
2,1.05,VVS1,E,None,Ideal,Jun 9,Jun 10,61.7,./diamond-details/LD07418539,Medium,...,RD,Round,LD07418539,False,NaN,Excellent,57.0,https://bluenile.v360.in/1/imaged/gia-12351614...,https://bnsec.bluenile.com/bnsecure/diamondvis...,False
3,1.35,VS2,G,None,Ideal,Jun 15,Jun 16,62.1,./diamond-details/LD09783992,None,...,RD,Round,LD09783992,False,NaN,Excellent,58.0,https://bluenile.v360.in/67/imaged/gia-7278979...,https://bnsec.bluenile.com/bnsecure/diamondvis...,False
4,1.40,VVS2,H,None,Ideal,Jul 15,Jul 16,62.7,./diamond-details/LD13516656,None,...,RD,Round,LD13516656,False,NaN,Excellent,58.0,https://bluenile.v360.in/50/imaged/gia-2346880...,https://bnsec.bluenile.com/bnsecure/diamondvis...,False


In [9]:
df.loc[12880, :]

carat                                                                 1.82
clarity                                                               VVS2
color                                                                    D
culet                                                                 None
cut                                                                  Ideal
date                                                                Jun 15
dateSet                                                             Jun 16
depth                                                                 60.8
detailsPageUrl                                ./diamond-details/LD13826785
fluorescence                                                          None
hasVisualization                                                      True
id                                                              LD13826785
imageUrl                                                               NaN
lxwRatio                 

In [35]:
# Diamond fluorescence itself is a debated topic, see https://www.leibish.com/diamond-fluorescence-article-245
# [TODO] Explore how different pair of color + fluorescence may result in different price
# Now we just simplify based on UV light intensity
print(df['fluorescence'].unique())
def map_fluorescence(x):
    if x == 'None':
        return 'None'
    elif 'Faint' in x:
        return 'Faint'
    elif 'Medium' in x:
        return 'Medium'
    elif 'Very Strong' in x:
        return 'Very Strong'
    elif 'Strong' in x:
        return 'Strong'
    else:
        raise ValueError('Unexpected value')
df['fluorescence_reduced'] = df['fluorescence'].map(map_fluorescence)
print(df['fluorescence_reduced'].unique())

['None' 'Medium' 'Faint' 'Strong Blue' 'Medium Blue' 'Strong'
 'Very Strong' 'Very Strong Blue' 'Medium Yellow']
['None' 'Medium' 'Faint' 'Strong' 'Very Strong']


In [36]:
# parse out measurement columns
measurements = df['measurements'].str.replace(' mm', '').str.split(' x ', expand=True)
measurements = measurements.apply(pd.to_numeric)
measurements.columns = ['length', 'width', 'height']
df = df.join(measurements)

In [37]:
# preprocess categorical variables
# shapeCode = RD based on condition when we download data from bluenile
# almost all of these variables are ordinal, ie. one category is more preferable than the other category
# we transform these oridnal variables as interval data
cols_cat = ['clarity', 'color',  'cut', 'culet', 'polish', 'symmetry', 'fluorescence_reduced']

# ranking: the bigger the better
ranking = {
    'clarity': ['I2', 'I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL'],
    'color': ['K', 'J', 'I', 'H', 'G', 'F', 'E', 'D'],
    'cut': ['Good', 'Very Good', 'Ideal', 'Astor Ideal'], # https://www.bluenile.com/education/diamonds/cut
    'culet': ['Medium', 'Small', 'Very Small', 'Pointed', 'None'],
    'polish': ['Good', 'Very Good', 'Excellent'],
    'symmetry': ['Good', 'Very Good', 'Excellent'],
    'fluorescence_reduced': ['Very Strong', 'Strong', 'Medium', 'Faint', 'None']
}

# ordinal encoding thru pandas
for cc in cols_cat:
    categories = pd.Categorical(df[cc], categories=ranking[cc], ordered=True)
    labels, unique = pd.factorize(categories, sort=True)
    df['cat_'+cc] = labels

In [38]:
df['hasVisualization'] = df['hasVisualization'].astype(int)

In [39]:
cols_num = ['carat', 'depth', 'lxwRatio', 'table', 'sellingIndex', 'hasVisualization', 'length', 'width', 'height']

dv = ['price']
iv = cols_num + ['cat_'+cc for cc in cols_cat]

In [40]:
df_process = df[dv + iv]
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14705 entries, 0 to 14704
Data columns (total 17 columns):
price                       14705 non-null int64
carat                       14705 non-null float64
depth                       14705 non-null float64
lxwRatio                    14705 non-null float64
table                       14705 non-null float64
sellingIndex                14705 non-null float64
hasVisualization            14705 non-null int64
length                      14705 non-null float64
width                       14705 non-null float64
height                      14705 non-null float64
cat_clarity                 14705 non-null int64
cat_color                   14705 non-null int64
cat_cut                     14705 non-null int64
cat_culet                   14705 non-null int64
cat_polish                  14705 non-null int64
cat_symmetry                14705 non-null int64
cat_fluorescence_reduced    14705 non-null int64
dtypes: float64(8), int64(9)
memory usage:

In [41]:
df_process.head()

,price,carat,depth,lxwRatio,table,sellingIndex,hasVisualization,length,width,height,cat_clarity,cat_color,cat_cut,cat_culet,cat_polish,cat_symmetry,cat_fluorescence_reduced
0,10001,1.50,59.9,1.01,60.0,0.191319,1,7.45,7.40,4.44,1,3,2,4,2,2,4
1,10002,1.51,62.7,1.01,57.0,0.333796,0,7.32,7.26,4.57,2,3,2,4,2,2,4
2,10004,1.05,61.7,1.00,57.0,0.193977,1,6.52,6.49,4.02,5,6,2,4,2,2,2
3,10005,1.35,62.1,1.01,58.0,0.245716,1,7.07,7.03,4.38,2,4,2,4,2,2,4
4,10005,1.40,62.7,1.01,58.0,0.561808,1,7.06,7.12,4.44,4,3,2,4,2,2,4


In [42]:
df_process.describe()

,price,carat,depth,lxwRatio,table,sellingIndex,hasVisualization,length,width,height,cat_clarity,cat_color,cat_cut,cat_culet,cat_polish,cat_symmetry,cat_fluorescence_reduced
count,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000,14705.000000
mean,17032.758177,1.831152,62.056307,1.007108,57.966066,0.260637,0.711595,7.769472,7.769338,4.820979,2.843047,4.157293,1.839238,3.887657,1.961850,1.892690,3.398300
std,4316.882473,0.399186,1.195928,0.004762,1.755220,0.179514,0.453036,0.553215,0.555594,0.357458,1.727052,1.987091,0.420176,0.521857,0.197164,0.332811,0.968497
min,10001.000000,0.900000,55.300000,1.000000,52.000000,0.000870,0.000000,6.060000,6.110000,1.510000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14074.000000,1.520000,61.600000,1.000000,57.000000,0.115079,0.000000,7.370000,7.370000,4.570000,1.000000,3.000000,2.000000,4.000000,2.000000,2.000000,3.000000
50%,16338.000000,1.820000,62.300000,1.010000,58.000000,0.217766,1.000000,7.800000,7.800000,4.850000,3.000000,4.000000,2.000000,4.000000,2.000000,2.000000,4.000000
75%,19655.000000,2.010000,62.700000,1.010000,59.000000,0.375305,1.000000,8.090000,8.100000,5.040000,4.000000,6.000000,2.000000,4.000000,2.000000,2.000000,4.000000
max,29994.000000,4.030000,67.900000,1.030000,68.000000,0.918382,1.000000,10.190000,10.190000,6.390000,7.000000,7.000000,3.000000,4.000000,2.000000,2.000000,4.000000


In [43]:
df_process.corr()

,price,carat,depth,lxwRatio,table,sellingIndex,hasVisualization,length,width,height,cat_clarity,cat_color,cat_cut,cat_culet,cat_polish,cat_symmetry,cat_fluorescence_reduced
price,1.000000,0.561932,-0.032942,-0.027882,0.023863,-0.140817,0.045430,0.578290,0.574658,0.545720,0.050617,0.017542,-0.013526,-0.001551,-0.000405,0.005302,-0.028636
carat,0.561932,1.000000,0.120878,0.037067,0.076483,-0.082008,-0.015909,0.981486,0.982872,0.977004,-0.416317,-0.598683,-0.137072,0.143189,-0.004255,-0.062950,-0.185433
depth,-0.032942,0.120878,1.000000,0.247921,-0.542552,0.021396,-0.024891,-0.019229,-0.009710,0.253331,-0.093847,-0.044348,-0.206772,0.129590,0.050529,-0.168894,-0.090623
lxwRatio,-0.027882,0.037067,0.247921,1.000000,-0.035417,0.035493,-0.048956,0.000234,0.004438,0.069746,-0.012351,-0.015322,-0.112013,0.089575,-0.048686,-0.190646,-0.049756
table,0.023863,0.076483,-0.542552,-0.035417,1.000000,0.058221,0.015402,0.142100,0.139653,-0.010212,0.018636,-0.091461,-0.128568,0.077584,-0.071933,-0.031963,0.007231
sellingIndex,-0.140817,-0.082008,0.021396,0.035493,0.058221,1.000000,0.356902,-0.077871,-0.079098,-0.069592,0.381764,0.000729,0.207833,0.118725,0.087345,0.033354,-0.381190
hasVisualization,0.045430,-0.015909,-0.024891,-0.048956,0.015402,0.356902,1.000000,-0.007882,-0.023647,-0.022907,0.052097,0.072757,0.239091,-0.103398,0.019190,0.000855,-0.042906
length,0.578290,0.981486,-0.019229,0.000234,0.142100,-0.077871,-0.007882,1.000000,0.995669,0.954046,-0.416804,-0.593882,-0.087702,0.131897,-0.008876,-0.026090,-0.168437
width,0.574658,0.982872,-0.009710,0.004438,0.139653,-0.079098,-0.023647,0.995669,1.000000,0.956528,-0.416314,-0.598180,-0.100610,0.144378,-0.009990,-0.031408,-0.171082
height,0.545720,0.977004,0.253331,0.069746,-0.010212,-0.069592,-0.022907,0.954046,0.956528,1.000000,-0.424633,-0.584070,-0.144754,0.165972,0.013142,-0.071484,-0.188311


In [44]:
df_process.to_csv('./data/diamonds_20200602_preprocess.csv', index=False)

In [32]:
# Split to train and test data
dataset_date = '20200620'

work_dir = '~/WorkDocs/Code/diamonds/data/{}'.format(dataset_date)
raw_path = os.path.join(work_dir, 'preprocess.csv')
df = pd.read_csv(raw_path)
df.head()

,price,carat,depth,lxwRatio,table,sellingIndex,hasVisualization,length,width,height,cat_clarity,cat_color,cat_cut,cat_culet,cat_polish,cat_symmetry,cat_fluorescence_reduced
0,10001,1.50,59.9,1.01,60.0,0.191319,1,7.45,7.40,4.44,1,3,2,4,2,2,4
1,10002,1.51,62.7,1.01,57.0,0.333796,0,7.32,7.26,4.57,2,3,2,4,2,2,4
2,10004,1.05,61.7,1.00,57.0,0.193977,1,6.52,6.49,4.02,5,6,2,4,2,2,2
3,10005,1.35,62.1,1.01,58.0,0.245716,1,7.07,7.03,4.38,2,4,2,4,2,2,4
4,10005,1.40,62.7,1.01,58.0,0.561808,1,7.06,7.12,4.44,4,3,2,4,2,2,4


In [33]:
SEED = 42
np.random.seed(SEED)

m = df.shape[0]
TRAIN_PERCENT = 0.8

indices = np.random.permutation(m)
cut = int(m * TRAIN_PERCENT)

train, test = df[:cut], df[cut:]
print("Training set shape: {}".format(train.shape))
print("Testing set shape: {}".format(test.shape))

Training set shape: (11764, 17)
Testing set shape: (2941, 17)


In [34]:
# Write to csv
train_path = os.path.join(work_dir, 'train.csv'.format(dataset_date))
test_path = os.path.join(work_dir, 'test.csv'.format(dataset_date))

train.to_csv(train_path, header=True, index=False)
test.to_csv(test_path, header=True, index=False)